<a href="https://colab.research.google.com/github/kabilan942/Natural-Language-Processing/blob/main/Research-Work/Legal/Transformers/LDS_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.21.0
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os

documentPath = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/'
dir = os.listdir(documentPath)

dir_list = []
for i in range(len(dir)):
  dir_list.append(int(dir[i][:-4]))

In [ ]:
"""
def read_input():
    file = open("D:/Phd_work/Dataset/Legal dataset/dataset/IN-Abs/train-data/judgement/6.txt", "r")
    FileContent = file.read().strip() # Stripping the white spaces in the file
    file = open("D:/Phd_work/Dataset/Legal dataset/dataset/IN-Abs/train-data/summary/6.txt", "r")
    summaryContent = file.read().strip()
    return(FileContent,summaryContent)
"""

# BART - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
FileContent,summaryContent=read_input()

In [ ]:
def model_load():
    checkpoint = "sshleifer/distilbart-cnn-12-6" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 125.9 MB/s eta 0:00:00


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Token indices sequence length is longer than the specified maximum sequence length for this model (2634 > 1024). Running this sequence through the model will result in indexing errors


tokenizer.model_max_length : 1024
 max tokens excluding the special tokens 1022
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 137
Total no of chunks created 3
Total no of tokens generated by chunks 2572
Total no of token in original text 2634
No of tokens in each  chunk excluding the special token [999, 1019, 548]


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print(final_summ_gen)

 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

tokenizer.model_max_length : 1024

max tokens excluding the special tokens 1022

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 137

Total no of chunks created 3

Total no of tokens generated by chunks 2572

Total no of token in original text 2634

No of tokens in each  chunk excluding the special token [999, 1019, 548]



{'rouge-1': {'r': 0.291044776119403, 'p': 0.20855614973262032, 'f': 0.24299064934191253},

'rouge-2': {'r': 0.08673469387755102, 'p': 0.049132947976878616, 'f': 0.06273062268923388},

'rouge-l': {'r': 0.26119402985074625, 'p': 0.18716577540106952, 'f': 0.21806853096185025}}

# BART - Chunking (Multiple Documents)

In [ ]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "sshleifer/distilbart-cnn-12-6" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list[:30]:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
len(result_rouge_list)

30

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5146350977636011, 'p': 0.2939724380970604, 'f': 0.3647993139364114}, 'rouge-2': {'r': 0.293680200989159, 'p': 0.1405336410499587, 'f': 0.18339711797892372}, 'rouge-l': {'r': 0.4749069546202089, 'p': 0.26922998613889076, 'f': 0.3350146321923683}}


# BigBird - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
from transformers import BigBirdForCausalLM

def model_load():
    checkpoint = "google/bigbird-roberta-base" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = BigBirdForCausalLM.from_pretrained(checkpoint,is_decoder=True)
#Model Details
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
from rouge import Rouge
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we reco

tokenizer.model_max_length : 4096
 max tokens excluding the special tokens 4094
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 138
Total no of chunks created 1
Total no of tokens generated by chunks 2571
Total no of token in original text 2636
No of tokens in each  chunk excluding the special token [2569]
N: Criminal Appeal No. 8 of 1951. Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950 arising out of an order dated 9th January, 1950, of the Presidency Magistrate, 19th Court, Esplanade, Bombay, in Case No. 10879/P of 1949. The facts are stated in the judgment. Iswarlal C. Dalai and R.B. Dalai, for the appellant. C.K. Daphtary, Solicitor General for India (G. N. Joshi, with him) for the Republic of India (respondent No. 1).Jindra Lal for the respondent No. 2. 1952. February 1. The Judgment of the Court was deliv ered by CHANDRASEKHARA AIYAR J. The facts out of which this Crimi 

If you want to use `BigBirdForCausalLM` as a standalone, add `is_decoder=True.`

tokenizer.model_max_length : 4096

max tokens excluding the special tokens 4094

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 138

Total no of chunks created 1

Total no of tokens generated by chunks 2571

Total no of token in original text 2636

No of tokens in each  chunk excluding the special token [2569]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
**Input length of input_ids is 2571, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.**

{'rouge-1': {'r': 0.22176308539944903, 'p': 0.8609625668449198, 'f': 0.35268345785982924},

'rouge-2': {'r': 0.15072830905636478, 'p': 0.6878612716763006, 'f': 0.2472727243240533},

'rouge-l': {'r': 0.22038567493112948, 'p': 0.8556149732620321, 'f': 0.35049287735599577}}

# BigBird - Chunking (Multiple Documents)

In [ ]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
from transformers import BigBirdForCausalLM

def model_load():
    checkpoint = "google/bigbird-roberta-base" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = BigBirdForCausalLM.from_pretrained(checkpoint,is_decoder=True)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list[:30]:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we reco

In [ ]:
len(result_rouge_list)

30

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.29704686623815646, 'p': 0.8199397745193063, 'f': 0.4288183742015664}, 'rouge-2': {'r': 0.18563868603051248, 'p': 0.651633184874414, 'f': 0.28231233567068204}, 'rouge-l': {'r': 0.28884331556801657, 'p': 0.7956824084144307, 'f': 0.41679113893456854}}


# BERT - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
from transformers import BertTokenizerFast
from transformers import EncoderDecoderModel

def model_load():
    checkpoint = "bert-base-uncased" #model loaded from Huggingface
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token
    model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")
    model.config.vocab_size = model.config.decoder.vocab_size
    model.config.max_length = 142
    model.config.min_length = 56
    model.config.no_repeat_ngram_size = 3
    model.config.early_stopping = True
    model.config.length_penalty = 2.0
    model.config.num_beams = 4

#Model Details
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
import nltk
#nltk.download('punkt')
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Token indices sequence length is longer than the specified maximum sequence length for this model (2665 > 512). Running this sequence through the model will result in indexing errors


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 510
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 147
Total no of chunks created 6
Total no of tokens generated by chunks 2675
Total no of token in original text 2665
No of tokens in each  chunk excluding the special token [489, 487, 494, 493, 488, 212]


ValueError: ignored

In [ ]:
print(final_summ_gen)

 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

tokenizer.model_max_length : 1024

max tokens excluding the special tokens 1022

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 137

Total no of chunks created 3

Total no of tokens generated by chunks 2572

Total no of token in original text 2634

No of tokens in each  chunk excluding the special token [999, 1019, 548]



{'rouge-1': {'r': 0.291044776119403, 'p': 0.20855614973262032, 'f': 0.24299064934191253},

'rouge-2': {'r': 0.08673469387755102, 'p': 0.049132947976878616, 'f': 0.06273062268923388},

'rouge-l': {'r': 0.26119402985074625, 'p': 0.18716577540106952, 'f': 0.21806853096185025}}

# T5 Small - Chunking (Single Document)
T5 Small

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/232.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/232.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "t5-small" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1
860
maximum tokens in longest sentence: 198
Total no of chunks created 78


Token indices sequence length is longer than the specified maximum sequence length for this model (37431 > 512). Running this sequence through the model will result in indexing errors


Total no of tokens generated by chunks 37508
Total no of token in original text 37431
No of tokens in each  chunk excluding the special token [470, 502, 505, 511, 500, 500, 471, 494, 503, 472, 465, 483, 507, 511, 491, 492, 502, 475, 474, 410, 473, 473, 456, 478, 487, 456, 475, 447, 497, 472, 488, 502, 506, 509, 476, 492, 504, 435, 509, 484, 501, 463, 499, 506, 486, 511, 505, 492, 492, 505, 502, 421, 479, 488, 494, 468, 480, 498, 494, 480, 452, 490, 494, 499, 445, 482, 423, 506, 498, 449, 496, 371, 499, 502, 509, 503, 445, 246]
{'rouge-1': {'r': 0.42317380352644834, 'p': 0.27495908346972175, 'f': 0.3333333285586932}, 'rouge-2': {'r': 0.16604708798017348, 'p': 0.09824046920821114, 'f': 0.1234454121877156}, 'rouge-l': {'r': 0.380352644836272, 'p': 0.24713584288052373, 'f': 0.29960316982853447}}


In [ ]:
print(final_summ_gen)

of the Court of Appeal, arising out of an order dated 20th February,,,, and the keys of the flat and the motor garage were handed overthe appellant was in search of a partner to carry on his business. thea tion for the relinquishment of his tenancy of anyis not a lawful transaction; it cannot be a unilateral transaction. of a tenancy is equivalent to surrender by the lessee or tenant of his


In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.

For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.

- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.

- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.

- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(

Token indices sequence length is longer than the specified maximum sequence length for this model (2859 > 512). Running this sequence through the model will result in indexing errors

tokenizer.model_max_length : 512

max tokens excluding the special tokens 511

num_special_tokens_to_add  1  98

maximum tokens in longest sentence: 147

Total no of chunks created 6

Total no of tokens generated by chunks 2864

Total no of token in original text 2859

No of tokens in each  chunk excluding the special token [502, 498, 398, 496, 489, 475]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.

{'rouge-1': {'r': 0.5686274509803921, 'p': 0.15508021390374332, 'f': 0.24369747562424976},

'rouge-2': {'r': 0.31343283582089554, 'p': 0.06069364161849711, 'f': 0.10169491253604121},

'rouge-l': {'r': 0.47058823529411764, 'p': 0.12834224598930483, 'f': 0.2016806689015607}}

# T5 Small - Chunking (Multiple Documents)

In [ ]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "t5-small" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk

        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk

        # reset the parameter to add next tokens to next chunk
            length = 0
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))


    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

In [ ]:
len(result_rouge_list)

100

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5340070793670784, 'p': 0.15040051883404526, 'f': 0.2259315977833998}, 'rouge-2': {'r': 0.2703121170814129, 'p': 0.052595345376596325, 'f': 0.08455800139639953}, 'rouge-l': {'r': 0.4836286813566721, 'p': 0.13425102682810058, 'f': 0.20248698625877598}}


# CLS

## Testing on Multiple Docs

In [ ]:
import pandas as pd
import numpy as np
import glob
import json
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')
from sklearn.metrics.pairwise import cosine_similarity
import torch

In [ ]:
# Loading Model and tokenizer
!pip install transformers
from transformers import AutoTokenizer
from transformers import  BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = False)
bert_model.eval()

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Single"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


2
2
2


In [ ]:
def get_sen_encoding(sents):
    '''
    Function to generate encoding for each word in the input list
    input: sents - List of sentences
    returns the list of the sentence encoding
    '''
    a = 0.001
    answer = None
    for sent in sents:
        ip =tokenizer(sent, return_tensors='pt', max_length=150, truncation=True, padding='max_length')  # to convert input text into integer ids
        tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])  # converts id into tokens with inclusion of special tokens added during tokenization such as [CLS]
        bert_output = bert_model(**ip)  # BertModel;
        embedding = bert_output['pooler_output'].clone().detach()
        if answer == None:
            answer = embedding
            answer.resize_(1, 768)
        else:
            embedding.resize_(1, 768)
            answer = torch.cat((answer, embedding),0)
    return answer

In [ ]:
def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary
    '''
    l = l1+l2  # list of sentences in both summary and document
    sents_encodings = get_sen_encoding(l)   # return a list of tensors of length 768 each, for each element of the list
    similarities=cosine_similarity(sents_encodings)   # calculates pairwise similiarities with elements in list if 2 arguments are not passed in; notice along diagonal its almost always equal to 1; https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity

    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [ ]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

In [ ]:
def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries
    '''
    chunk_summ_word_threshold = 150
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)

    result = (similarity_l_l(summ_sents,doc_sents))

    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]

    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 1024):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(tokenizer.tokenize(summ)) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

In [ ]:
import pandas as pd
training_chunks = []
training_summs = []
for i in tqdm(range(2)):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
#     print(i, len(training_summs), end = ", ", sep = " : ")
    if i%100 == 0:
        full = list(zip(training_chunks,training_summs))
        df = pd.DataFrame(full,columns=['data', 'summary'])
        df.to_excel("FD_CLS_BK.xlsx")
#         break
full = list(zip(training_chunks,training_summs))
df = pd.DataFrame(full,columns=['data', 'summary'])
df.to_excel("FD_CLS.xlsx")

100%|██████████| 2/2 [24:39<00:00, 739.53s/it]


In [ ]:
df.head()

,data,summary
0,The learned Chief Justice rejected the argumen...,(vi) The notification issued under the Act in ...
1,"This passage, which was relied on by the learn...","Per FAZL ALl, MAHAJAN, MUKHERJEA and CHANDRASE..."
2,It was held that the equal protection clause c...,5 (1) clearly and unambigu ously vests the Sta...
3,"The meaning, scope and effect of 334 article 1...",5(1) which empowers the State Government to di...
4,N: Criminal Appeal No. 8 of 1951. Appeal from ...,Sub section (1) of sec.(2) provided that any t...


In [ ]:
df.shape[0]

6

### BART

In [ ]:
def model_load():
    checkpoint = "sshleifer/distilbart-cnn-12-6" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def sum_generate(sent,tokenizer,model):
    inputs = tokenizer(sent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in range(df.shape[0]):
    FileContent,summaryContent=df['data'][i],df['summary'][i]
    tokenizer,model=model_load()
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
result_rouge_list

[{'rouge-1': {'r': 0.1956521739130435,
   'p': 0.10227272727272728,
   'f': 0.1343283537001561},
  'rouge-2': {'r': 0.10344827586206896,
   'p': 0.05172413793103448,
   'f': 0.06896551279693515},
  'rouge-l': {'r': 0.1956521739130435,
   'p': 0.10227272727272728,
   'f': 0.1343283537001561}},
 {'rouge-1': {'r': 0.525, 'p': 0.21, 'f': 0.2999999959183674},
  'rouge-2': {'r': 0.3269230769230769,
   'p': 0.11724137931034483,
   'f': 0.17258882860161312},
  'rouge-l': {'r': 0.45, 'p': 0.18, 'f': 0.25714285306122453}},
 {'rouge-1': {'r': 0.45454545454545453,
   'p': 0.2702702702702703,
   'f': 0.3389830461706407},
  'rouge-2': {'r': 0.34615384615384615,
   'p': 0.1565217391304348,
   'f': 0.21556885798702005},
  'rouge-l': {'r': 0.45454545454545453,
   'p': 0.2702702702702703,
   'f': 0.3389830461706407}},
 {'rouge-1': {'r': 0.30120481927710846,
   'p': 0.32894736842105265,
   'f': 0.31446540381472265},
  'rouge-2': {'r': 0.10810810810810811,
   'p': 0.11538461538461539,
   'f': 0.1116279019

In [ ]:
len(result_rouge_list)

12

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.3634682475938811, 'p': 0.17569865824076542, 'f': 0.22943430770090015}, 'rouge-2': {'r': 0.17294006518912516, 'p': 0.07011826138793202, 'f': 0.09680838176568345}, 'rouge-l': {'r': 0.31536742920716243, 'p': 0.15548464048749114, 'f': 0.20146606568101716}}


### T5 Small

In [ ]:
def model_load():
    checkpoint = "t5-small" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def sum_generate(sent,tokenizer,model):
    inputs = tokenizer(sent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in range(df.shape[0]):
    FileContent,summaryContent=df['data'][i],df['summary'][i]
    tokenizer,model=model_load()
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
result_rouge_list

[{'rouge-1': {'r': 0.5, 'p': 0.022727272727272728, 'f': 0.0434782600378072},
  'rouge-2': {'r': 0.25, 'p': 0.008620689655172414, 'f': 0.016666666022222245},
  'rouge-l': {'r': 0.5, 'p': 0.022727272727272728, 'f': 0.0434782600378072}},
 {'rouge-1': {'r': 0.4, 'p': 0.02, 'f': 0.03809523718820863},
  'rouge-2': {'r': 0.2, 'p': 0.006896551724137931, 'f': 0.01333333268888892},
  'rouge-l': {'r': 0.4, 'p': 0.02, 'f': 0.03809523718820863}},
 {'rouge-1': {'r': 0.5, 'p': 0.0945945945945946, 'f': 0.15909090641528928},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.5, 'p': 0.0945945945945946, 'f': 0.15909090641528928}},
 {'rouge-1': {'r': 0.5, 'p': 0.07894736842105263, 'f': 0.13636363400826448},
  'rouge-2': {'r': 0.07692307692307693,
   'p': 0.009615384615384616,
   'f': 0.017094015118708682},
  'rouge-l': {'r': 0.5, 'p': 0.07894736842105263, 'f': 0.13636363400826448}},
 {'rouge-1': {'r': 0.8461538461538461,
   'p': 0.09016393442622951,
   'f': 0.1629629612224966},
  'rouge-2'

In [ ]:
len(result_rouge_list)

12

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.44649794649794655, 'p': 0.04393028452240499, 'f': 0.0783279222204719}, 'rouge-2': {'r': 0.1483974358974359, 'p': 0.007057754027624917, 'f': 0.013333931528360942}, 'rouge-l': {'r': 0.4265595515595516, 'p': 0.041888345616817216, 'f': 0.07469491337654495}}


## Testing for Single Doc

In [ ]:
FileContent,summaryContent=df['data'][0],df['summary'][0]
tokenizer,model=model_load()

NameError: ignored

In [ ]:
inputs = tokenizer(FileContent, return_tensors="pt",max_length=1024,truncation=True)
inputs

{'input_ids': tensor([[   0,  133, 2435,  ...,   50, 1200,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
len(inputs['input_ids'])

1

In [ ]:
output = model.generate(inputs['input_ids'])
output

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    2,     0,  1231,  1659,  3946,  4795,    14,     5,  2136,    22,
         28162,   113,    11,     5,  2849,  2810,   197,     6,    11,  1217,
             9,     5,  1270,     8,  1198,   424,  5225,     9,     5,  1783,
             6,    28, 37139,    25,  3099,  1200,  7980,  2078,   906,  1500,
           479,    91,   303,    24,   113,  4703,     7,   847,   159,     5,
         10798,  3099,     9,     5,  2136,   128, 28162,   128,    25,   341,
            11,     5,  2810,   108,     2]])

In [ ]:
summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output
summ_generated

' Chief Justice rejected argument that the word "cases" in the sub section should, in view of the title and preamble of the Act, be construed as meaning cases requiring speedier trial. He found it" impossible to cut down the plain meaning of the word \'cases\'as used in the section\''

In [ ]:
len(summ_generated)

280

In [ ]:
rouge_score(summaryContent,summ_generated)

{'rouge-1': {'r': 0.43243243243243246,
  'p': 0.0893854748603352,
  'f': 0.1481481453090707},
 'rouge-2': {'r': 0.20454545454545456,
  'p': 0.031578947368421054,
  'f': 0.05471124388355624},
 'rouge-l': {'r': 0.2972972972972973,
  'p': 0.061452513966480445,
  'f': 0.10185184901277443}}

In [ ]:
chunk_summ_word_threshold = 150
sentence_mapping = {}
doc_sents = split_to_sentences(data_source[0])

In [ ]:
sum_sent = split_to_sentences(data_summary[0])

In [ ]:
len(sum_sent)

18

In [ ]:
len(doc_sents)

98

In [ ]:
doc_sents[:3]

['N: Criminal Appeal No.',
 '8 of 1951.',
 'Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.)']

In [ ]:
sent = 'Appeal from an Order of the High Court of Bombay Bavdekar and Chainani JJ.'
ip =tokenizer(sent, return_tensors='pt', max_length=150, truncation=True, padding='max_length')  # to convert input text into integer ids
ip

{'input_ids': tensor([[  101, 13969,  1121,  1126,  2864,  1104,  1103,  1693,  2031,  1104,
         11888, 18757,  1964, 27170,  1813,  1105, 19589,  7192,   147,  4538,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])  # converts id into tokens with inclusion of special tokens added during tokenization such as [CLS]
tokens

['[CLS]',
 'Appeal',
 'from',
 'an',
 'Order',
 'of',
 'the',
 'High',
 'Court',
 'of',
 'Bombay',
 'Ba',
 '##v',
 '##dek',
 '##ar',
 'and',
 'Chain',
 '##ani',
 'J',
 '##J',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',

In [ ]:
bert_output = bert_model(**ip)  # BertModel;
embedding = bert_output['pooler_output'].clone().detach()

In [ ]:
bert_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3768,  0.1182,  0.1627,  ...,  0.1455,  0.0282,  0.5325],
         [ 0.1971, -0.0953,  0.5898,  ..., -0.1703,  0.4177,  0.0018],
         [ 0.5735,  0.0626,  0.6084,  ..., -0.2785,  0.5612,  0.2554],
         ...,
         [ 0.2255, -0.0391,  0.3593,  ..., -0.3351,  0.1854,  0.0733],
         [ 0.2426, -0.1024,  0.2483,  ..., -0.3510,  0.2703,  0.1072],
         [-0.0915,  0.0703,  0.2405,  ..., -0.0438, -0.0511,  0.1174]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.8858e-01, -2.3746e-01, -6.8391e-02,  4.9617e-01, -2.6217e-01,
         -6.3700e-02,  7.4489e-01,  2.7868e-01,  2.7921e-01, -9.9988e-01,
          5.3527e-01,  2.1707e-02,  9.4602e-01,  6.9972e-02,  7.0043e-01,
         -4.6001e-01,  2.0929e-01, -4.3320e-01,  2.1176e-01, -3.7752e-01,
          4.3815e-01,  9.9335e-01,  3.0895e-01,  1.6129e-01,  2.3410e-01,
         -3.7454e-02, -3.0041e-01,  7.6712e-01,  8.9378e-01,  6.710

In [ ]:
len(bert_output[0][0])

150

In [ ]:
answer=None
if answer == None:
    answer = embedding
    answer.resize_(1, 768)
else:
    embedding.resize_(1, 768)
    answer = torch.cat((answer, embedding),0)

In [ ]:
answer

tensor([[-6.8858e-01, -2.3746e-01, -6.8391e-02,  4.9617e-01, -2.6217e-01,
         -6.3700e-02,  7.4489e-01,  2.7868e-01,  2.7921e-01, -9.9988e-01,
          5.3527e-01,  2.1707e-02,  9.4602e-01,  6.9972e-02,  7.0043e-01,
         -4.6001e-01,  2.0929e-01, -4.3320e-01,  2.1176e-01, -3.7752e-01,
          4.3815e-01,  9.9335e-01,  3.0895e-01,  1.6129e-01,  2.3410e-01,
         -3.7454e-02, -3.0041e-01,  7.6712e-01,  8.9378e-01,  6.7108e-01,
         -5.7889e-01,  2.0379e-01, -9.6417e-01, -1.8043e-01, -5.8530e-01,
         -9.6328e-01,  1.9576e-01, -4.8963e-01, -9.0488e-02,  6.0243e-03,
         -7.2304e-01,  2.6920e-01,  9.9856e-01, -1.3884e-01, -9.6088e-02,
         -2.0296e-01, -9.9940e-01,  1.7848e-01, -7.0146e-01, -5.4707e-02,
         -1.8947e-01, -3.4721e-01,  3.6112e-02,  2.9797e-01,  3.0387e-01,
          2.3301e-01, -1.3490e-01,  1.7855e-01, -2.6691e-01, -4.1850e-01,
         -4.9815e-01,  3.2025e-01, -2.3150e-01, -7.6433e-01, -4.6488e-01,
         -9.8679e-02, -7.0457e-02, -3.

In [ ]:
similarities=cosine_similarity(answer)
similarities

array([[1.]], dtype=float32)

# Fine-tuning BART using CLS

In [ ]:
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.5/816.5 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492036 sha256=59fe809e4ccc4986c58f1863962870296892c221efd65801dfff68ee0d2a8f86
  Stored in directory: /root/.cache/pip/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Un

In [ ]:
import pytorch_lightning as pl
print(pl.__version__)

1.2.2


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
filename = "/content/FD_CLS.xlsx"

data = pd.read_excel(filename,index_col=0)
data.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)

In [ ]:
data.shape

(6, 2)

In [ ]:
data

,source,target
0,The learned Chief Justice rejected the argumen...,(vi) The notification issued under the Act in ...
1,"This passage, which was relied on by the learn...","Per FAZL ALl, MAHAJAN, MUKHERJEA and CHANDRASE..."
2,It was held that the equal protection clause c...,5 (1) clearly and unambigu ously vests the Sta...
3,"The meaning, scope and effect of 334 article 1...",5(1) which empowers the State Government to di...
4,N: Criminal Appeal No. 8 of 1951. Appeal from ...,Sub section (1) of sec.(2) provided that any t...
5,The section which consists of two parts is in ...,"19 of the Bombay Rents, Hotel and Lodging Hous..."


In [ ]:
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, df, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.data = df

  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels'])
    val_data = DataLoader(dataset, batch_size = self.batch_size)
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels'])
    test_data = DataLoader(dataset, batch_size = self.batch_size)
    return test_data

In [ ]:
def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=1024, min_length = 512, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=min_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)


  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch

In [ ]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    # self.freeze_encoder = freeze_encoder
    # self.freeze_embeds_ = freeze_embeds
#     self.hparams = argparse.Namespace()

    self.hparams.freeze_encoder = True
    self.hparams.freeze_embeds = True
    self.hparams.eval_beams = 4
    # self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()

  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}

  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 1024):
    ''' Function to generate text '''
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = self.tokenizer.pad_token_id,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

In [ ]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

In [ ]:
def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

Embedding(50272, 1024)

In [ ]:
summary_data = SummaryDataModule(tokenizer, data, batch_size = 1)

In [ ]:
summary_data

In [ ]:
model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [ ]:
!pip install torch==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 1.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


In [ ]:
import torch.fx

In [ ]:
trainer = pl.Trainer(max_epochs = 3,
                     min_epochs = 2,
                     auto_lr_find = False,
                     progress_bar_refresh_rate = 5,
                     precision = 16)

GPU available: False, used: False
GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


MisconfigurationException: ignored

In [ ]:
trainer.fit(model, summary_data)

In [ ]:
trainer.save_checkpoint("output.ckpt")

In [ ]:
!zip -r output.zip ./output.ckpt